In [1]:
from imblearn.over_sampling._smote.base import SMOTE
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pickle
 
class MyClass():
    def __init__(self, param):
        self.param = param
 
def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)

X_train = load_object("X_train.pickle")
X_test = load_object("X_test.pickle")
y_train = load_object("y_train.pickle")
y_test = load_object("y_test.pickle")

pipeline_estimators = [('scaling',StandardScaler()),('clf',LogisticRegression())]
pipe = Pipeline(pipeline_estimators)



The X_train,y_train,X_test,y_test were retrieved from a Pickle module and will now be used to train the model

In [2]:
X_test.head(2)

,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,Temperature,Wind_Chill,Humidity,Pressure,...,Thunder / Windy,Thunder and Hail / Windy,Thunder in the Vicinity,Thunderstorm,Thunderstorms and Rain,Tornado,Volcanic Ash,Widespread Dust,Wintry Mix,Wintry Mix / Windy
85699,40.70555,-74.24590,40.70375,-74.24677,0.132,1,81.0,51.852029,58.0,30.05,...,0,0,0,0,0,0,0,0,0,0
29466,32.92059,-96.75061,32.92033,-96.75013,0.033,1,80.6,51.852029,77.0,29.89,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn.metrics import accuracy_score


pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_pred,y_test)

0.8204355594822869

In [4]:
from collections import Counter

Counter(y_pred)

Counter({0.0: 150564, 1.0: 1103})

In [5]:
X_test.loc[[2]]

,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,Temperature,Wind_Chill,Humidity,Pressure,...,Thunder / Windy,Thunder and Hail / Windy,Thunder in the Vicinity,Thunderstorm,Thunderstorms and Rain,Tornado,Volcanic Ash,Widespread Dust,Wintry Mix,Wintry Mix / Windy
2,39.10266,-84.52468,39.10209,-84.52396,0.055,1,36.0,51.852029,97.0,29.7,...,0,0,0,0,0,0,0,0,0,0


Here below the prediction is tested to see if the output is the expected one

In [6]:

pred = pipe.predict(X_test.loc[[2]])

In [7]:
pred[0]

0.0

The model is saved into a file, it will be used for the prediction of new samples during the execution of the application

In [8]:
import joblib

joblib.dump(pipe, 'completed_model.joblib')

['completed_model.joblib']